# 🚚 배달 소요 시간 예측 데이터셋 명세 (Dataset Specification)

이 데이터셋은 주문 발생 시점부터 고객에게 음식이 도착하기까지의 **총 소요 시간(Delivery_Time_min)** 을 예측하기 위해 수집된 변수들로 구성되어 있습니다.

---

## 1. 타겟 변수 (Target Variable)

| 컬럼명 | 설명 | 단위 | 비고 |
|:---|:---|:---:|:---|
| **Delivery_Time_min** | 주문 후 고객에게 도착하기까지 총 소요 시간 | 분(min) | **예측 목표(Target)** |

---

## 2. 특징 변수 (Input Features)

### 📋 주문 및 상점 상태 (Order & Store)
* **Prep_Time_min**: 음식 조리 및 포장에 소요되는 시간 (분)
* **Order_Queue**: 해당 매장의 현재 주문 적체 건수 (건)
* **Promo**: 프로모션/할인 적용 여부 (0: 미적용, 1: 적용)
* **Tip_Level**: 고객이 설정한 팁 수준 (0 ~ 3 단계)

### 📍 거리 및 물류 (Distance & Logistics)
* **Distance_km**: 매장에서 배달지까지의 이동 거리 (km)
* **Riders_Available**: 주변에서 즉시 배차 가능한 라이더 수 (명)
* **Vehicle**: 배달 수단 (`Bike`, `Scooter`, `Car`)
* **Road_Type**: 도로 환경 구분 (`Downtown`, `Suburban`, `Highway`)

### ☁️ 기상 및 환경 (Weather & Environment)
| 컬럼명 | 설명 | 단위/값 |
|:---|:---|:---:|
| **Weather** | 날씨 상태 | 맑음, 비, 눈, 안개 |
| **Rain_mm** | 강수량 | mm |
| **Temp_C** | 현재 기온 | °C |
| **Wind_mps** | 풍속 | m/s |

### ⏰ 시간 정보 (Temporal)
* **Rush_Hour**: 교통 혼잡 시간대 여부 (0: 해당 없음, 1: 러시아워)
* **DayOfWeek**: 요일 정보 (0: 월요일 ~ 6: 일요일)
* **Time_Slot**: 하루 중 시간대 구분 (`Morning`, `Lunch`, `Afternoon`, `Dinner`, `Night`)

---

## 3. 데이터 전처리 가이드 (Pre-processing Notes)

1.  **Categorical Encoding**: `Weather`, `Time_Slot`, `Vehicle`, `Road_Type` 변수는 모델링 전 One-Hot Encoding 또는 Label Encoding이 필요합니다.
2.  **Outliers**: `Delivery_Time_min`이 비정상적으로 길거나 `Distance_km`가 지나치게 먼 데이터는 이상치 처리를 권장합니다.
3.  **Scaling**: 거리($km$)와 기온($^\circ C$) 등 단위 차이가 큰 변수들은 Standard Scaling 또는 Min-Max Scaling을 고려하십시오.

***

## 🤖 머신러닝 모델링 가이드

### 1) 문제 유형
* **유형**: 지도 학습 (Supervised Learning) - **회귀 (Regression)**
* **타겟(Y)**: `Delivery_Time_min` (연속형 변수)

### 2) 성능 평가 지표 (Evaluation Metrics)
회귀 모델이므로 다음 지표를 사용하여 오차를 측정합니다.
* **MAE (Mean Absolute Error)**: 실제 시간과 예측 시간의 평균 차이(분)
* **RMSE (Root Mean Squared Error)**: 큰 오차에 가중치를 둔 지표
* **R² Score (결정계수)**: 모델이 데이터를 얼마나 잘 설명하는지 (1에 가까울수록 좋음)

### 3) 추천 알고리즘
* Linear Regression (선형 회귀)
* Random Forest Regressor
* XGBoost / LightGBM Regressor (성능이 가장 우수할 가능성 높음)

---

### ✂️ 피처 셀렉션 (Feature Selection)
모델의 복잡도를 줄이고 성능을 최적화하기 위해 상관계수 절대값을 기준으로 변수를 선별함.

* **기준**: $|r| \ge 0.1$
* **선별된 변수**:
    * `Distance_km` (0.88)
    * `Prep_Time_min` (0.23)
    * `Weather_Clear` (-0.22) 등...
* **제외된 변수**: 상관계수가 0.1 미만인 변수들 (예: `Order_ID`, `Promo` 등)은 모델의 노이즈를 줄이기 위해 제거를 검토함.

***

### 🛠️ 시각화 가독성 개선 (EDA Optimization)

**문제점**: `Distance_km` 등의 수치형 변수를 Boxplot으로 시각화할 경우, X축에 너무 많은 범주가 생성되어 그래프가 검게 뭉치는 현상 발생.

**개선 사항**:
1. **변수 타입별 시각화 분리**: 
   - **수치형(Continuous)**: `regplot`을 통해 타겟 변수와의 선형 관계 및 데이터 밀도 확인.
   - **범주형(Categorical)**: `boxplot`을 통해 각 그룹별(예: 러시아워 여부) 평균 배달 시간 차이 확인.
2. **불필요한 그래프 제거**: 자기 자신과의 상관관계를 보여주는 타겟 vs 타겟 그래프 제외.

***

### 🌊 밀도 기반 시각화 (KDE Plot) 적용
데이터 포인트가 너무 많아 산점도(`regplot`)에서 오버플로팅(Overplotting) 현상이 발생함에 따라, 데이터의 밀집 지역을 파악하기 위해 **2D KDE Plot**을 도입함.

* **장점**: 
    - 점들의 겹침 현상 없이 거리(`km`) 대비 시간(`min`)의 집중 구간을 시각적으로 확인 가능.
    - 이상치(Outlier)를 제외한 데이터의 주된 흐름을 파악하기 용이함.
* **적용 대상**: `Distance_km`, `Prep_Time_min`, `Rain_mm` 등 수치형 변수.

***

### ⚠️ 트러블슈팅: ValueError: continuous is not supported

**1. 발생 상황**
* 모델 학습 후 성능 평가를 위해 `accuracy_score` 등을 호출하는 과정에서 발생.

**2. 원인 분석**
* **문제 유형 오류**: 배달 소요 시간 예측은 **회귀(Regression)** 문제이나, 평가 지표로 **분류(Classification)**용 지표(`Accuracy`, `Precision`, `Recall`)를 사용함.
* 분류 지표는 정답이 '0 또는 1'처럼 명확한 범주형일 때만 작동하며, 25.4분 같은 연속형 숫자에는 대응하지 못함.

**3. 해결 방법**
* 회귀 모델 전용 평가지표인 `MAE`, `RMSE`, `R2 Score`로 평가 로직을 교체함.
* 특히 **MAE**를 통해 실제 배달 시간과 평균적으로 몇 분 정도 오차가 발생하는지 직관적으로 파악함.

***

### 🏷️ 함수 네이밍 가이드 (Naming Convention)
| 함수 이름 | 권장 용도 | 특징 |
| :--- | :--- | :--- |
| `evaluate_regression_model` | 공식 문서/최종 코드 | 목적과 타겟이 가장 명확함 |
| `print_regression_metrics` | 로그 출력용 | 지표(Metrics)를 출력한다는 의미 강조 |
| `show_errors` | 디버깅용 | 오차(Error) 확인에 집중함 |

**선택한 이름**: `evaluate_regression_model`
**이유**: `accuracy_score` 등을 사용하는 분류(Classification) 평가 함수와 혼동을 피하기 위해 `regression` 키워드를 명시함.

# 실험 코드

In [ ]:
import pandas as pd

df_origin_train = pd.read_csv("../../data/delivery_eta_train.csv")
df_train = df_origin_train.copy()
df_origin_train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 상관관계 계산 시 의미 없는 컬럼(ID 등) 제외
corr_matrix = df_train.drop(columns=["Order_ID"]).corr()

# 2. 그래프 크기 대폭 확대 (변수 개수에 따라 조절)
plt.figure(figsize=(25, 18))

# 3. 히트맵 설정 최적화
sns.heatmap(
    corr_matrix,
    annot=True,  # 숫자 표시
    fmt=".2f",  # 소수점 둘째자리까지
    cmap="coolwarm",  # 컬러맵
    linewidths=0.5,  # 칸 사이 간격 (구분선)
    annot_kws={"size": 10},  # 숫자 글자 크기 키우기
    cbar_kws={"shrink": 0.8},
)  # 컬러바 크기 조절

plt.title("Delivery Time Feature Correlation", fontsize=20)
plt.xticks(rotation=45, ha="right")  # X축 라벨 45도 회전
plt.show()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cols = ["Weather", "Time_Slot", "Vehicle", "Road_Type"]
encoder = OneHotEncoder(sparse_output=False)

# 1. 인코딩 수행 (데이터프레임의 실제 데이터를 넣어야 함)
encoded_array = encoder.fit_transform(df_train[cols])

# 2. 새 컬럼명 생성 및 데이터프레임 변환
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(cols))

# 3. 기존 컬럼 삭제 후 합치기
df_train = pd.concat([df_train.drop(columns=cols), encoded_df], axis=1)
df_train

In [ ]:
corr_matrix = df_train.corr()

plt.figure(figsize=(20, 18))
sns.heatmap(corr_matrix, annot=True, cmap="RdBu", fmt=".2f")
plt.show()

In [ ]:
# 2. Delivery_Time_min과 다른 변수들 간의 상관계수만 추출
target_corr = corr_matrix[["Delivery_Time_min"]].sort_values(
    by="Delivery_Time_min", ascending=False
)

plt.figure(figsize=(8, 10))
sns.heatmap(target_corr, annot=True, fmt=".2f", cmap="RdBu", cbar=True)
plt.title("Correlation with Delivery Time")
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
target_corr["Delivery_Time_min"].plot(kind="barh", color="skyblue")
plt.title("Correlation of Features with Delivery Time")
plt.xlabel("Correlation Coefficient")
plt.axvline(x=0, color="black", linestyle="--", linewidth=1)  # 0 기준선
plt.show()

⚠️ Error Log: ValueError: Columns must be same length as key
* 발생 원인
  * 원-핫 인코딩(One-Hot Encoding)은 범주 개수에 따라 컬럼 수가 늘어나는데, 이를 원본 컬럼 개수(4개)와 동일한 공간에 강제로 대입하려고 시도함.
  * OneHotEncoder 결과물의 열(Column) 개수와 df[label_col]의 열 개수가 불일치하여 발생.

* 해결책
  * pd.get_dummies()를 사용하여 자동으로 컬럼을 확장하거나, pd.concat()을 사용하여 원본 데이터프레임에 새로운 인코딩 컬럼들을 결합함.


방법 1
```python
import pandas as pd

label_cols = ["Weather", "Time_Slot", "Vehicle", "Road_Type"]

# 원본 df에서 해당 컬럼들만 원-핫 인코딩하여 교체
df = pd.get_dummies(df, columns=label_cols)
```

방법 2
```python
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

label_cols = ["Weather", "Time_Slot", "Vehicle", "Road_Type"]
encoder = OneHotEncoder(sparse_output=False)

# 1. 인코딩 수행 (데이터프레임의 실제 데이터를 넣어야 함)
encoded_array = encoder.fit_transform(df[label_cols])

# 2. 새 컬럼명 생성 및 데이터프레임 변환
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(label_cols))

# 3. 기존 컬럼 삭제 후 합치기
df = pd.concat([df.drop(columns=label_cols), encoded_df], axis=1)
```

***

### 🤖 모델 선택 및 평가 전략 (Model & Evaluation)

**1. 올바른 모델 선택**
* 배달 시간 예측은 **회귀 문제**이므로 `RandomForestClassifier` 대신 **`RandomForestRegressor`**를 사용해야 함.

**2. 모델별 평가 지표 분리**
* **회귀 모델 (Regressor)**: `evaluate_regression_model` 함수 사용 (MAE, RMSE, R2).
* **분류 모델 (Classifier)**: `accuracy_score`, `confusion_matrix` 등을 사용하며, 현재 프로젝트(배달 시간)에는 부적합함.

***

### 🏆 모델 성능 비교 및 최종 선정
각 회귀 알고리즘의 예측력을 평가 지표(MAE, $R^2$)를 통해 비교 분석함.

* **성능 지표**: 
    - **MAE**: 실제 배달 시간과 예측값의 물리적 거리(분) 측정.
    - **$R^2$ Score**: 모델의 데이터 설명력을 백분율과 유사한 수치로 확인.
* **분석 결과**: 
    - `RandomForestRegressor`가 선형 회귀 대비 낮은 MAE를 기록하여 비선형 관계를 더 잘 포착함.
    - `XGBoost`는 대량의 데이터에서 가장 높은 $R^2$ 점수를 보여 최종 모델로 검토됨.
* **시각화**: 'Actual vs Predicted' 산점도를 통해 특정 시간대(예: 장거리 배달)에서 오차가 커지는 경향을 확인하고 보완함.

***

### 🔑 변수 중요도 분석 (Feature Importance Analysis)

모델(`RandomForestRegressor`)이 예측 시 어떤 변수를 중요하게 고려했는지 분석함.

* **최고 중요 변수**: `Distance_km`
    * 상관계수(0.88) 결과와 일치하게 모델 내에서도 압도적인 영향력을 보임.
* **주요 기여 변수**: `Prep_Time_min`, `Weather`, `Rush_Hour`
    * 물리적 거리 외에도 조리 시간과 환경적 요인이 배달 지연의 핵심 변수로 작용함.
* **인사이트**: 
    - 거리(`km`) 변수의 데이터 품질이 전체 모델 성능을 결정짓는 핵심 요소임.
    - `Rain_mm`나 `Weather` 같은 기상 변수 역시 유의미한 비중을 차지하므로, 악천후 시 배달 지연 보정 알고리즘에 활용 가능함.